In [8]:
import simpy

In [2]:
## First test

def car(env: simpy.Environment):
    while True:
        print(f'Start parking at {env.now}')
        parking_duration = 5
        yield env.timeout(parking_duration)

        print(f'Start driving at {env.now}')
        trip_duration = 2
        yield env.timeout(trip_duration)


env = simpy.Environment()
env.process(car(env))
env.run(until = 45)


Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14
Start driving at 19
Start parking at 21
Start driving at 26
Start parking at 28
Start driving at 33
Start parking at 35
Start driving at 40
Start parking at 42


In [6]:
## Process interaction, make a process wait on another.

class Car(object):

    def __init__(self,env: simpy.Environment):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
        while True:
            print(f'Start parking and charging at {env.now}')
            charge_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield env.process(self.charge(charge_duration))

            # The charge process has finished and
            # we can start driving again.
            print(f'Start driving at {env.now}')
            trip_duration = 2
            yield env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)

env = simpy.Environment()
myCar = Car(env)
env.run(until = 15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


In [7]:
## Interrupting another process. Impatient driver wants to interrupt charging

class Car(object):
     def __init__(self, env: simpy.Environment):
         self.env = env
         self.action = env.process(self.run())

     def run(self):
         while True:
             print('Start parking and charging at %d' % self.env.now)
             charge_duration = 5
             # We may get interrupted while charging the battery
             try:
                 yield self.env.process(self.charge(charge_duration))
             except simpy.Interrupt:
                 # When we received an interrupt, we stop charging and
                 # switch to the "driving" state
                 print('Was interrupted. Hope, the battery is full enough ')

             print('Start driving at %d' % self.env.now)
             trip_duration = 2
             yield self.env.timeout(trip_duration)

     def charge(self, duration):
         yield self.env.timeout(duration)

def driver(env: simpy.Environment, car: Car):
     yield env.timeout(3)
     car.action.interrupt()

env = simpy.Environment()
myCar = Car(env)
env.process(driver(env,myCar))
env.run(until = 15)


Start parking and charging at 0
Was interrupted. Hope, the battery is full enough 
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


In [12]:
## Usage of shared resources, definately useful.

# This time, the car will drive to a battery-charging station: bcs

def car(env: simpy.Environment, name, bcs:simpy.Resource, driving_time, charge_duration):
    #Driving to the bcs.
    env.timeout(driving_time)

    # Request a spot
    print(f'{name} arriving at {env.now}')
    with bcs.request() as req:
        yield req

        # Charge battery
        print(f'{name} Starting to charge at {env.now}')
        yield env.timeout(charge_duration)
        print(f'{name} leaving bcs at {env.now}')

env = simpy.Environment()
bcs = simpy.Resource(env,capacity = 2)

for i in range(4):
    env.process(car(env,f'Car {i}',bcs,i*2,5))

env.run(until=35)

Car 0 arriving at 0
Car 1 arriving at 0
Car 2 arriving at 0
Car 3 arriving at 0
Car 0 Starting to charge at 0
Car 1 Starting to charge at 0
Car 0 leaving bcs at 5
Car 1 leaving bcs at 5
Car 2 Starting to charge at 5
Car 3 Starting to charge at 5
Car 2 leaving bcs at 10
Car 3 leaving bcs at 10
